In [ ]:
import cv2
import mediapipe as mp
import numpy as np
import pickle
import time
import os
from flask import Flask, Response
import threading

mp_pose = mp.solutions.pose
mp_drawing = mp.solutions.drawing_utils

# Global frame for streaming
current_frame = None

RTSP stream opened successfully


## Helper: Calculate Angle

In [ ]:
def calculate_angle(a, b, c):
    a = np.array(a)
    b = np.array(b)
    c = np.array(c)
    ab = a - b
    bc = c - b
    cosine_angle = np.dot(ab, bc) / (np.linalg.norm(ab) * np.linalg.norm(bc) + 1e-6)
    angle = np.arccos(np.clip(cosine_angle, -1.0, 1.0))
    return np.degrees(angle)

## Perform Exercise with RTSP Stream (RPi → Laptop Display)

In [ ]:
def perform_exercise_rtsp(exercise_name, laptop_ip):
    """
    Run on RPi: Captures from local webcam, processes poses, streams result to laptop via Flask.
    
    Args:
        exercise_name: Name of the .pkl reference file (e.g., 'squat')
        laptop_ip: Your laptop's IP to display in the print message
    """
    global current_frame
    
    pkl_path = f"{exercise_name}.pkl"
    if not os.path.exists(pkl_path):
        print(f"Reference not found: {pkl_path}. Upload it to RPi first!")
        return

    with open(pkl_path, 'rb') as f:
        reference_sequence = pickle.load(f)
    
    if not reference_sequence:
        print("Empty reference sequence.")
        return

    pose = mp_pose.Pose()
    
    # Use RPi local camera (not RTSP - we're creating the stream FROM the Pi)
    cap = cv2.VideoCapture(0)
    
    ref_index = 0
    animation_fps = 10
    frame_delay = 1.0 / animation_fps
    last_ref_time = time.time()
    angle_threshold = 20
    
    pose_connections = mp_pose.POSE_CONNECTIONS
    
    print(f"Starting {exercise_name} on RPi.")
    print(f"View stream on laptop: http://{laptop_ip}:5000/video")
    print("(Make sure Flask server is running - see next cell)")

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            print("Cannot read from camera!")
            break
        
        height, width, _ = frame.shape
        rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        results = pose.process(rgb)
        
        black = np.zeros((height, width, 3), dtype=np.uint8)
        
        # Draw reference (BLUE)
        ref_lm = reference_sequence[ref_index]
        for conn in pose_connections:
            start, end = ref_lm[conn[0]], ref_lm[conn[1]]
            if start[3] > 0.1 and end[3] > 0.1:
                cv2.line(black,
                         (int(start[0] * width), int(start[1] * height)),
                         (int(end[0] * width), int(end[1] * height)),
                         (255, 0, 0), 2)
        
        # Draw user (GREEN if correct, RED if wrong)
        if results.pose_landmarks:
            user_lm = [(lm.x, lm.y, lm.z, lm.visibility) for lm in results.pose_landmarks.landmark]
            
            for conn in pose_connections:
                i1, i2 = conn
                r1, r2 = ref_lm[i1], ref_lm[i2]
                u1, u2 = user_lm[i1], user_lm[i2]
                
                if all(x[3] > 0.1 for x in [r1, r2, u1, u2]):
                    parent_idx = None
                    for c in pose_connections:
                        if c[1] == i2 and c[0] != i1:
                            parent_idx = c[0]; break
                        if c[0] == i2 and c[1] != i1:
                            parent_idx = c[1]; break
                    
                    if parent_idx is not None and ref_lm[parent_idx][3] > 0.1 and user_lm[parent_idx][3] > 0.1:
                        rp, up = ref_lm[parent_idx], user_lm[parent_idx]
                        
                        ref_angle = calculate_angle((r1[0], r1[1]), (r2[0], r2[1]), (rp[0], rp[1]))
                        user_angle = calculate_angle((u1[0], u1[1]), (u2[0], u2[1]), (up[0], up[1]))
                        
                        color = (0, 255, 0) if abs(ref_angle - user_angle) < angle_threshold else (0, 0, 255)
                        
                        cv2.line(black,
                                 (int(u1[0] * width), int(u1[1] * height)),
                                 (int(u2[0] * width), int(u2[1] * height)),
                                 color, 2)
        
        # Encode frame for streaming
        current_frame = cv2.imencode('.jpg', black)[1].tobytes()
        
        # Advance reference animation
        if time.time() - last_ref_time >= frame_delay:
            last_ref_time = time.time()
            ref_index = (ref_index + 1) % len(reference_sequence)
        
        time.sleep(0.01)  # Small delay to prevent CPU overload
    
    cap.release()
    print("Exercise stopped.")

## Start Flask Streaming Server (Run on RPi)

In [ ]:
app = Flask(__name__)

@app.route('/')
def index():
    return '<html><body><h2>RPi Workout Stream</h2><img src="/video" width="100%"></body></html>'

def gen():
    global current_frame
    while True:
        if current_frame is not None:
            yield (b'--frame\r\n'
                   b'Content-Type: image/jpeg\r\n\r\n' + current_frame + b'\r\n')
        time.sleep(0.01)

@app.route('/video')
def video_feed():
    return Response(gen(), mimetype='multipart/x-mixed-replace; boundary=frame')

def run_server():
    app.run(host='0.0.0.0', port=5000, threaded=True, use_reloader=False)

# Start Flask in background
thread = threading.Thread(target=run_server, daemon=True)
thread.start()
print("Flask server running on RPi at port 5000")
print("Access from laptop: http://[RPI-IP]:5000")

## Run the Exercise (Update IPs and exercise name)

In [ ]:
# 1. Upload your .pkl reference file to the RPi (e.g., squat.pkl)
# 2. Update the IPs below
# 3. Run this cell

LAPTOP_IP = "192.168.0.36"  # Your laptop's IP (to show the stream URL)
EXERCISE_NAME = "squat"      # Name of your .pkl file (without extension)

# Start the exercise
perform_exercise_rtsp(EXERCISE_NAME, LAPTOP_IP)